In [23]:
# Importa as dependências
import numpy as np
import cv2

In [24]:
# Traduz uma imagem do domínio espacial para o da frequência 
def space_to_freq (image):

    # Converte a imagem para floats e aplica o DFT, 
    # obtendo os valores complexos
    dft = cv2.dft(np.float32(image), flags = cv2.DFT_COMPLEX_OUTPUT)

    # realiza um 'shift' do canto superior esquerdo  
    # para o centro da matrix na frequência
    dft_shift = np.fft.fftshift(dft)

    # extrai a magnetude e fase da matrix na frequência
    mag, phase = cv2.cartToPolar(dft_shift[:,:,0], dft_shift[:,:,1])

    # traduz de magnetude
    # para aspectro 
    spec = np.log(mag) / 20

    # retorna os valores calculados
    return spec, mag, phase

In [25]:
# Traduz uma imagem do domínio da frequência para o espacial
def freq_to_space (spec, mag, phase, img_min, img_max):

    # traduz do aspectro 
    # para magnetude
    mag = np.exp(spec * 20)

    # converte a magentude e fase para 
    # valores reais e complexos no plano carteziano
    real, imag = cv2.polarToCart(mag, phase)

    # combina os componentes carteziados 
    # em uma matrix de valores complexos
    back = cv2.merge([real, imag])

    # realiza um 'shift' do centro 
    # para o canto superior esquerdo
    back_ishift = np.fft.ifftshift(back)

    # realiza um idft, 
    # salva complexos resultantes
    img_back = cv2.idft(back_ishift)

    # combina os componentes complexos de volta para a imagem
    img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])

    # re-normaliza para o range uint8, igual a original
    min, max = np.amin(img_back, (0,1)), np.amax(img_back, (0,1))
    image = cv2.normalize(img_back, None, alpha=img_min, 
                            beta=img_max, norm_type=cv2.NORM_MINMAX, 
                            dtype=cv2.CV_8U)

    # retorna a imagem
    return image

In [26]:
# mouse callback function
pt1_x, pt1_y, drawing = 0, 0, 0
def line_drawing(event, x, y, flags, param):
    
    # Puxa algumas variáveis globais para execução
    global sidebyside, spectrum_image, pt1_x, pt1_y, drawing
    color = 0
    
    # Ao pressionar, atualiza status e var locais
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        pt1_x, pt1_y = x, y

    # Ao mover, pinta uma linha do último frame ao atual
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.line(spectrum_image, (pt1_x, pt1_y), (x, y), color=(color, color, color), thickness=5)
            pt1_x, pt1_y = x, y
            
    # Ao soltar realiza uma última atualização
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.line(spectrum_image, (pt1_x, pt1_y), (x, y), color=(color, color, color), thickness=5) 

In [27]:
# Lê a imagem em BW
image = cv2.imread('image.jpg', 0)

# Obtêm os valores de min e max da imagem
img_min, img_max = np.amin(image, (0, 1)), np.amax(image, (0, 1))

# Traduz para o domínio da frequência
spectrum_image, mag, phase = space_to_freq(image)
saved = spectrum_image

# Cria a janela e seta o callback
cv2.namedWindow('Spectrum')
cv2.setMouseCallback('Spectrum', line_drawing)

while(1):

    # Realiza um update na imagem resultante
    freq_image = freq_to_space(spectrum_image, mag, phase, 
                               img_min, img_max)

    # Concatena a imagem modificada 
    # com sua versão na frequência
    sidebyside = np.concatenate((spectrum_image, freq_image / 255), axis=1)       

    # Mostra a imagem montada
    cv2.imshow('Spectrum', sidebyside)

    # Para a execução caso Q pressionado
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Reseta as edições caso R pressionado
    elif cv2.waitKey(1) & 0xFF == ord('r'):
        spectrum_image, mag, phase = space_to_freq(image)

# Fecha as janelas
cv2.destroyAllWindows()